# Проблема затухающего градиента в НС 
Рассмотрим задачу где есть:
- ***НС*** с 4 слоями
- 1 слой - 1 нейрон
- Формула ***НС*** $ f = a_4(w_4*a_3(w_3*a_2(w_2*a_1(w_1*x))))$ 

     где:
     * $f$ - выход из ***НС***
     * $x$ - входной скаляр 
     * $w_n$ - веса $n$-ного слоя
     * $a_n$ - нелинейная функция активации на $n$-ном слое
- Требуется найти градиент по всем $w_n$

     если:
     * $a_n = Tanh()$ 
     * $a_n = ReLU()$ 
---

## Разложим формулу и применим правило цепочки: 

$f_1 = tanh(w_1*x)$

$f_2 = tanh(w_2*f_1)$

$f_3 = tanh(w_3*f_2)$

$f = tanh(w_4*f_3)$

$\frac{d(f)}{d(w_4)} = \frac{d(f)}{d(f_3)} * f_3$

$\frac{d(f)}{d(w_3)} = \frac{d(f)}{d(f_3)} * \frac{d(f_3)}{d(f_2)} * f_2$

$\frac{d(f)}{d(w_2)} = \frac{d(f)}{d(f_3)} * \frac{d(f_3)}{d(f_2)} * \frac{d(f_2)}{d(f_1)} * f_1$

$\frac{d(f)}{d(w_1)} = \frac{d(f)}{d(f_3)} * \frac{d(f_3)}{d(f_2)} * \frac{d(f_2)}{d(f_1)} * \frac{d(f_1)}{d(x)} * x$

$tanh'(x) = 1 - tanh^2(x)$

---
## Решение


### Для начала импортируем $tanh$ 

In [1]:
from numpy import tanh

---
## Инициализируем $x$ и $w$

In [2]:
x = 100
w_1, w_2, w_3, w_4 = [1,1,1,1]

In [3]:
f_1 = tanh(w_1*x)
f_2 = tanh(w_2*f_1)
f_3 = tanh(w_3*f_2)

In [4]:
grad_w4 = (1-tanh(f_3)**2)*f_3
grad_w3 = (1-tanh(f_3)**2)*(1-tanh(f_2)**2)*f_2
grad_w2 = (1-tanh(f_3)**2)*(1-tanh(f_2)**2)*(1-tanh(f_1)**2)*f_1
grad_w1 = (1-tanh(f_3)**2)*(1-tanh(f_2)**2)*(1-tanh(f_1)**2)*(1-tanh(x)**2)*x

---
## Создадим функцию для вывода информации

In [7]:
def show(fn_act,grads_w):
    print(fn_act)
    for n, grad in enumerate(grads_w):
        print(f"\tГрадиент W на {n+1} слое:\t{grad}")

---
## Выведем информацию

In [8]:
show("Tanh:",(grad_w1,grad_w2,grad_w3,grad_w4))

Tanh:
	Градиент W на 1 слое:	0.0
	Градиент W на 2 слое:	0.16770685876939032
	Градиент W на 3 слое:	0.3041246830975466
	Градиент W на 4 слое:	0.436145382444544


---
## Теперь посчитаем это с помощью PyTorch-autograd

In [9]:
import torch
from torch.nn import Tanh,ReLU

In [10]:
sclr = lambda v: torch.tensor(v,requires_grad=True)
x = sclr(100.)
w_1, w_2, w_3, w_4 =  [sclr(1.) for _ in range(4)]

In [11]:
for a in [Tanh(),ReLU()]:
    f = lambda x: a(w_4 * a(w_3 * a(w_2 * a(w_1 * x))))
    y = f(x)
    y.backward()
    show(a,(w_1.grad,w_2.grad,w_3.grad,w_4.grad))
    for var in [w_1.grad,w_2.grad,w_3.grad,w_4.grad]:
        var.zero_()

Tanh()
	Градиент W на 1 слое:	0.0
	Градиент W на 2 слое:	0.16770686209201813
	Градиент W на 3 слое:	0.30412471294403076
	Градиент W на 4 слое:	0.4361453950405121
ReLU()
	Градиент W на 1 слое:	100.0
	Градиент W на 2 слое:	100.0
	Градиент W на 3 слое:	100.0
	Градиент W на 4 слое:	100.0


 ## Вывод
 Из данных наблюдений следует, что функция активации $Tanh$, которая активно
 
 применялась в LeNet(1998), в некоторых ситуациях, сильно способствует затуханию градиента.
 
 Что в свою очередь, делает почти что, невозможным её примение в глубоких ***НС***.